<a href="https://colab.research.google.com/github/Somesh0001/ML-Translations/blob/main/Indic_Translations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git
%%capture
%cd /content/IndicTrans2/huggingface_interface
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransToolkit.git
%cd IndicTransToolkit
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..
%%capture
!pip install indic-nlp-library

In [73]:
!pip install streamlit -q
import os
from threading import Thread
from pyngrok import ngrok
from pyngrok import ngrok
ngrok.set_auth_token("2dRr6-enter-your-auth-token-here-Xj6J8")

In [74]:
print(os.getcwd())

/content/IndicTrans2/huggingface_interface/IndicTransToolkit


In [75]:
%%writefile app.py
import sys
import os
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer

# Add the directory containing your IndicTransToolkit module to the Python path
# Assuming your notebook is in the IndicTrans2 folder

print(os.getcwd())
sys.path.append(os.path.join(os.getcwd(),  "IndicTransToolkit"))

# Now try importing IndicProcessor
from processor import IndicProcessor

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()
    model.eval()

    return tokenizer, model

def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations


import streamlit as st
st.title("# Indic Language Translator ")
st.write("## Translate between indian languages and foreign languages ")
input_language = st.selectbox('Select the input language', ['English', 'Hindi', 'Spanish', 'French', 'German'])
output_language = st.selectbox('Select the output language', ['English', 'Hindi', 'Spanish', 'French', 'German'])
input_text = st.text_input(f"Write the sentence in {input_language} and it will be translated to {output_language}")

en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

en_sents = [
  input_text
]

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
st.write(f"Translation in {output_language}: ")
for input_sentence, translation in zip(en_sents, hi_translations):
    st.write(f" {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model


Overwriting app.py


In [76]:
def run_streamlit():
    # Change the port if 8501 is already in use or if you prefer another port
    os.system('streamlit run /content/IndicTrans2/huggingface_interface/IndicTransToolkit/app.py --server.port 8501')
    # Start a thread to run the Streamlit app
thread = Thread(target=run_streamlit)
thread.start()
# Open a tunnel to the streamlit port 8501
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print('Your Streamlit app is live at:', public_url)

Your Streamlit app is live at: NgrokTunnel: "https://304d-35-232-58-145.ngrok-free.app" -> "http://localhost:8501"


In [67]:
ngrok.kill()